In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pymongo import MongoClient
from pandas.io.json import json_normalize

plt.style.use('ggplot')
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']  #解决seaborn中文字体显示问题
plt.rc('figure', figsize=(10, 10))  #把plt默认的图片size调大一点
plt.rcParams["figure.dpi"] =mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
%matplotlib inline

In [400]:
#设置中文标签的显示
plt.rcParams['font.sans-serif'] = ['SimHei']  #设置显示中文字体
#读取csv中的数据，以“,”作为分隔符来添加字段名
data = pd.read_csv('F:/kriswu.csv', delimiter=',', encoding='utf8',dtype={'user.follow_count':int,'user.followers_count':int,"comments_count":int,"attitudes_count":int,
                 "reposts_count":int,"user.mbrank":int},keep_default_na=False)

In [330]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102118 entries, 0 to 102117
Data columns (total 20 columns):
attitudes_count           102118 non-null int32
comments_count            102118 non-null int32
reposts_count             102118 non-null int32
mid                       102118 non-null int64
raw_text                  102118 non-null object
source                    98487 non-null object
user.description          79118 non-null object
user.follow_count         102118 non-null int32
user.followers_count      102118 non-null int32
user.gender               102118 non-null object
user.id                   102118 non-null int64
user.mbrank               102118 non-null int32
user.mbtype               102118 non-null int64
user.profile_url          102118 non-null object
user.profile_image_url    102118 non-null object
user.screen_name          102118 non-null object
user.statuses_count       102118 non-null int64
user.urank                102118 non-null int64
user.verified       

In [331]:
data.sample(5)

,attitudes_count,comments_count,reposts_count,mid,raw_text,source,user.description,user.follow_count,user.followers_count,user.gender,user.id,user.mbrank,user.mbtype,user.profile_url,user.profile_image_url,user.screen_name,user.statuses_count,user.urank,user.verified,user.verified_reason
96137,1,0,0,4362915827472379,我凡牛批哦💗,HUAWEI nova 2s,無謝幕的人生,49,20,f,6510092048,0,0,https://m.weibo.cn/u/6510092048?uid=6510092048,https://tvax2.sinaimg.cn/crop.0.0.996.996.180/...,一顆檸檬味少女,9,9,False,NaN
39157,0,0,0,4362830222413002,有点儿意思[哈哈][哈哈][哈哈]//@王思聪:歌真的挺不错，而且凡凡也很man。[加油],OPPO随光而变R17,NaN,440,126,f,5173013048,0,0,https://m.weibo.cn/u/5173013048?uid=5173013048,https://tvax2.sinaimg.cn/crop.0.0.996.996.180/...,立志减肥的微博,1474,9,False,NaN
49378,0,0,0,4362800853532934,可爱好听,小米8 SE,熊猫酱 ⊙▽⊙,633,56,f,2898179111,0,0,https://m.weibo.cn/u/2898179111?uid=2898179111,https://tvax2.sinaimg.cn/crop.0.0.1002.1002.18...,快去追啊绿芽菜,2258,9,False,NaN
19541,1,0,0,4362784290712638,过两年cxk发个自嘲唱跳rap打篮球的视频也一样变风向吸粉 大部分人对娱乐明星其实是非常宽容...,iPhone客户端,嫉饿如仇|颜良|文丑,247,191,m,5674175425,0,0,https://m.weibo.cn/u/5674175425?uid=5674175425,https://tvax3.sinaimg.cn/crop.0.0.512.512.180/...,夜之川诗衣奈,721,4,False,NaN
94232,0,0,0,4362838505939348,//@炼狱蜕变重生:明天演唱会不见不散//@韩束旗舰店:凡凡喊妮来见面啦[给你小心心][给你...,iPhone客户端,"Taking my time,step by step *敢想敢做*",504,243,f,2159568374,2,2,https://m.weibo.cn/u/2159568374?uid=2159568374,https://tvax3.sinaimg.cn/crop.0.0.1080.1080.18...,-Cheryl2,755,30,False,NaN


#### 1. 数据清洗
由于数据入库的时候没有进行清洗，所以数据多出了很多没用的字段，需要先清洗掉

In [332]:
print(list(data.columns))

['attitudes_count', 'comments_count', 'reposts_count', 'mid', 'raw_text', 'source', 'user.description', 'user.follow_count', 'user.followers_count', 'user.gender', 'user.id', 'user.mbrank', 'user.mbtype', 'user.profile_url', 'user.profile_image_url', 'user.screen_name', 'user.statuses_count', 'user.urank', 'user.verified', 'user.verified_reason']


In [333]:
in_columns = ['attitudes_count', 'comments_count', 'reposts_count', 'mid', 'raw_text', 
          'source', 'user.description', 'user.follow_count', 'user.followers_count', 
          'user.gender', 'user.id', 'user.mbrank', 'user.mbtype', 'user.profile_url', 
          'user.profile_image_url', 'user.screen_name', 'user.statuses_count', 
          'user.urank', 'user.verified', 'user.verified_reason']

In [334]:
data = data[in_columns]

In [335]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102118 entries, 0 to 102117
Data columns (total 20 columns):
attitudes_count           102118 non-null int32
comments_count            102118 non-null int32
reposts_count             102118 non-null int32
mid                       102118 non-null int64
raw_text                  102118 non-null object
source                    98487 non-null object
user.description          79118 non-null object
user.follow_count         102118 non-null int32
user.followers_count      102118 non-null int32
user.gender               102118 non-null object
user.id                   102118 non-null int64
user.mbrank               102118 non-null int32
user.mbtype               102118 non-null int64
user.profile_url          102118 non-null object
user.profile_image_url    102118 non-null object
user.screen_name          102118 non-null object
user.statuses_count       102118 non-null int64
user.urank                102118 non-null int64
user.verified       

In [385]:
data.to_csv('F:/kriswu.csv', index=False)

问题：
1. 吴亦凡该微博的转发是否存在假流量？
2. 大家对于《大碗宽面》怎么看？
3. 有多少人拿吴亦凡跟蔡徐坤做对比？
4. 有多少人开始路转粉了？
5. 评论的词云图

### 1. 吴亦凡该微博的转发是否存在假流量？

In [386]:
# 先来看看吴亦凡的粉丝性别比例
fans_num = data['user.gender'].value_counts()
fans_num

f    77279
m    24839
Name: user.gender, dtype: int64

In [387]:
from pyecharts import Bar

In [388]:


bar = Bar("吴亦凡粉丝性别比例初探", width =600,height=500)
bar.add("(总数据102118条)", ['女', '男'], fans_num.values, is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

In [389]:
np.round(fans_num/fans_num.sum()*100, 2)

f    75.68
m    24.32
Name: user.gender, dtype: float64

In [390]:
data[data['user.gender']=='f'].sample(5)

,attitudes_count,comments_count,reposts_count,mid,raw_text,source,user.description,user.follow_count,user.followers_count,user.gender,user.id,user.mbrank,user.mbtype,user.profile_url,user.profile_image_url,user.screen_name,user.statuses_count,user.urank,user.verified,user.verified_reason
5037,0,0,0,4362787330803939,哭了[跪了]//@M鹿M:这碗面儿你干了，我随意[耶][耶][耶],华为手机 畅享玩不停,想要努力奋斗加油的学生。,172,75,f,6126518383,1,2,https://m.weibo.cn/u/6126518383?uid=6126518383,https://tvax4.sinaimg.cn/crop.0.0.996.996.180/...,卧幺控哒闲,81,8,False,NaN
101865,0,0,0,4362895040658734,哥哥生日快乐[心]//@M鹿M:这碗面儿你干了，我随意[耶][耶][耶],OPPO R9s,明天的日出还是要期盼的吧,107,231,f,6593271510,3,2,https://m.weibo.cn/u/6593271510?uid=6593271510,https://tvax4.sinaimg.cn/crop.0.0.996.996.180/...,唐鱼是恋爱单排选手,130,9,False,NaN
57574,0,0,0,4362815210785099,转发微博,iPhone客户端,转发微博。,1939,657,f,1751098234,0,0,https://m.weibo.cn/u/1751098234?uid=1751098234,https://tva2.sinaimg.cn/crop.0.15.750.750.180/...,双味小酥,17729,43,False,NaN
62417,0,0,0,4362899617153876,转发微博,Android,平安喜乐,403,137,f,3493161775,1,2,https://m.weibo.cn/u/3493161775?uid=3493161775,https://tva1.sinaimg.cn/crop.0.0.1068.1068.180...,YY小豆包儿,15165,14,False,NaN
27961,0,0,0,4362828993369751,现在的哥哥都有趣,HUAWEI nova 2 Plus,冷库追星沙雕🤘,336,112,f,6293744458,0,0,https://m.weibo.cn/u/6293744458?uid=6293744458,https://tvax2.sinaimg.cn/crop.0.0.996.996.180/...,云鹤九霄龙腾四海可以让我挑吗,848,9,False,NaN


In [391]:
data[data['user.gender']=='m'].sample(5)

,attitudes_count,comments_count,reposts_count,mid,raw_text,source,user.description,user.follow_count,user.followers_count,user.gender,user.id,user.mbrank,user.mbtype,user.profile_url,user.profile_image_url,user.screen_name,user.statuses_count,user.urank,user.verified,user.verified_reason
69964,0,0,0,4362890625023678,不是，他怎么好像还真是自己作词作曲啊，有点不行啊[允悲][允悲]//@穿胖次的狐狸:[并不简...,Share微博客户端,预备役社畜,284,238,m,3969693338,1,2,https://m.weibo.cn/u/3969693338?uid=3969693338,https://tvax1.sinaimg.cn/crop.0.0.640.640.180/...,简单熊23333,5063,14,False,NaN
58704,0,0,0,4362904276513795,转发微博,小米8 屏幕指纹版,吧唧,58,32,m,5647195226,1,2,https://m.weibo.cn/u/5647195226?uid=5647195226,https://tvax3.sinaimg.cn/crop.0.0.1002.1002.18...,kabuto911,18,14,False,NaN
21363,0,0,0,4362812144910436,这波公关团队的操作非常的及时，和某cai的操作瞬间甩来了一万米,Android,Trust no chigga,331,84,m,5235020311,0,0,https://m.weibo.cn/u/5235020311?uid=5235020311,https://tvax2.sinaimg.cn/crop.0.0.996.996.180/...,chrisweihbbb,2310,14,False,NaN
39118,0,0,0,4362791689697983,吴亦凡对8起,iPhone客户端,NaN,171,136,m,2436796630,0,0,https://m.weibo.cn/u/2436796630?uid=2436796630,https://tva3.sinaimg.cn/crop.0.11.489.489.180/...,蓝恩伯特,200,23,False,NaN
16074,0,0,0,4362796772518213,?爱你一[舔屏]万年，是我的追求[钟]~[坏笑]！[兔子]#黄子韬[超话]#[给力],微博 HTML5 版,NaN,0,1,m,6810359413,0,0,https://m.weibo.cn/u/6810359413?uid=6810359413,https://tvax2.sinaimg.cn/default/images/defaul...,用户6810359413,67,4,False,NaN


In [401]:
data_fake = data[((data['user.follow_count']<=5)|(data['user.followers_count']<=5))&
                 (data['user.description']=='')&
                 (data['comments_count']==0)&
                (data['attitudes_count']==0)&
                (data['reposts_count']==0)&
                (data['user.mbrank']==0)]
data_fake.sample(5)

,attitudes_count,comments_count,reposts_count,mid,raw_text,source,user.description,user.follow_count,user.followers_count,user.gender,user.id,user.mbrank,user.mbtype,user.profile_url,user.profile_image_url,user.screen_name,user.statuses_count,user.urank,user.verified,user.verified_reason
9022,0,0,0,4362796487145464,在纯真年代别太多感慨，把一切[奥特曼]热爱尽情的盛开[礼物]~[女孩儿]！[浪]#黄子韬[超...,微博 HTML5 版,,0,1,m,6791117524,0,0,https://m.weibo.cn/u/6791117524?uid=6791117524,https://tvax1.sinaimg.cn/default/images/defaul...,用户6791117524,159,4,False,
32830,0,0,0,4362797258823266,寶貝、你想我吗、我今天一天沒有看到你、做什麽[熊猫]事情都沒有精神、你說怎麽辦阿[污]~[赞...,微博 HTML5 版,,0,1,m,6741745243,0,0,https://m.weibo.cn/u/6741745243?uid=6741745243,https://tvax4.sinaimg.cn/default/images/defaul...,用户6741745243,383,4,False,
16732,0,0,0,4362796748171766,最好的那天太难挑选，择日不[舔屏]如撞日[广告]~[鲜花]！[晕]#黄子韬[超话]#[作揖],微博 HTML5 版,,0,1,f,6743123379,0,0,https://m.weibo.cn/u/6743123379?uid=6743123379,https://tvax4.sinaimg.cn/default/images/defaul...,用户6743123379,369,4,False,
57464,0,0,0,4362807871503378,//@M鹿M:这碗面儿你干了，我随意[耶][耶][耶],iPhone客户端,,68,1,m,6876326037,0,0,https://m.weibo.cn/u/6876326037?uid=6876326037,https://tvax2.sinaimg.cn/default/images/defaul...,肖申克的舅叔8,23,4,False,
33321,0,0,0,4362796914807450,不得不承认，吴亦凡是真的比我帅了那么一点，歌嘛。。。,iPhone客户端,,5,12,m,1870910102,0,0,https://m.weibo.cn/u/1870910102?uid=1870910102,https://tvax1.sinaimg.cn/crop.0.2.957.957.180/...,树伯伯_,9,4,False,


In [403]:
data_fake.shape

(5667, 20)

In [404]:
# 昵称里包含“用户”的，基本上可以断定是假粉丝
data_fake2_index = data[(data['user.follow_count']>5)&
                        (data['user.followers_count']>5)&
                        (data['user.screen_name'].str.contains('用户'))].index

In [405]:
# 把假的流量粉丝转发组合起来
data_fake = pd.concat([data_fake, data.iloc[data_fake2_index]])

In [406]:
data_fake.shape

(6100, 20)

In [407]:
# 取出真粉的转发
data_true = data.drop(data_fake.index)

In [408]:
data_true.shape

(96018, 20)

In [409]:
print('真粉丝转发XQ数占总转发数的{}%'.format(np.round(data_true.shape[0]/data.shape[0]*100, 2)))
print('假粉丝转发数占总转发数的{}%'.format(np.round(data_fake.shape[0]/data.shape[0]*100, 2)))

真粉丝转发XQ数占总转发数的94.03%
假粉丝转发数占总转发数的5.97%


In [410]:
bar = Bar("吴亦凡真假流量的转发量", width = 600,height=500)
bar.add("(总数据102118条)", ['总转发量', '假粉丝转发量', '真粉丝转发量'], 
        [data.shape[0], data_fake.shape[0], data_true.shape[0]], is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

In [411]:
real_fans_num = data_true.drop_duplicates(subset='user.id').shape[0]

In [412]:
bar = Bar("吴亦凡真假流量的转发量与真实转发粉丝量(总数据102118条)", width = 600,height=500)
bar.add('', ['总转发量', '假粉丝转发量', '真粉丝转发量', '真实转发粉丝量'], 
        [data.shape[0], data_fake.shape[0], data_true.shape[0], real_fans_num], is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True, xaxis_rotate=20)
bar

In [413]:
print('真实转发粉丝量占总转发数的{}%'.format(np.round(real_fans_num/data.shape[0]*100, 2)))

真实转发粉丝量占总转发数的80.17%


### 2. 大家对于《大碗宽面》怎么看？

In [414]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102118 entries, 0 to 102117
Data columns (total 20 columns):
attitudes_count           102118 non-null int32
comments_count            102118 non-null int32
reposts_count             102118 non-null int32
mid                       102118 non-null int64
raw_text                  102118 non-null object
source                    102118 non-null object
user.description          102118 non-null object
user.follow_count         102118 non-null int32
user.followers_count      102118 non-null int32
user.gender               102118 non-null object
user.id                   102118 non-null int64
user.mbrank               102118 non-null int32
user.mbtype               102118 non-null int64
user.profile_url          102118 non-null object
user.profile_image_url    102118 non-null object
user.screen_name          102118 non-null object
user.statuses_count       102118 non-null int64
user.urank                102118 non-null int64
user.verified     

In [415]:
from snownlp import SnowNLP


def get_sent_snownlp(data):
    s = SnowNLP(data)
    return s.sentiments

In [416]:
data_true['clean_text'] = data_true['raw_text'].str.split("//", expand=True)[0]

In [417]:
data_true['sent_nlp'] = data_true.loc[data_true['clean_text']!='', 'clean_text'].apply(get_sent_snownlp)

In [418]:
data_true[['clean_text', 'sent_nlp']].sample(5)

,clean_text,sent_nlp
59594,走起[鲜花][haha][haha][haha],0.693529
41236,对比蔡徐坤的律师函 吴亦凡瞬间提了不少分,0.999688
97965,哥哥哥哥哥哥哥哥[悲伤][悲伤][悲伤][心][心][心],1.000000
64034,,NaN
88787,俺觉得凡凡意有所指[思考][偷笑],0.833975


In [419]:
data_true.loc[-data_true['clean_text'].isin(['转发微博', 'repost', '轉發微博']), 'sent_nlp'].mean()

0.6860448043677209

In [421]:
from pyecharts import Gauge, Page


g = Gauge()
g.add('', ['评论对《大碗宽面》\n的平均评分'], [68.6])
g

In [422]:
data_true.loc[data_true['sent_nlp']>0.9, ['clean_text', 'sent_nlp']].sample(5)

,clean_text,sent_nlp
50879,mv好可爱 哈哈哈哈哈哈哈哈哈,0.999271
74840,哥哥太棒了[haha],0.995762
12533,蔡徐坤好好学学。不过也是实力不到位，不然谁愿意做谐星来妥协呢[喵喵],0.995648
43880,我的牛桃全世界最甜[泪][泪][泪][心],0.999916
55405,最后一段man炸了 洒脱面对质疑真是酷[乾杯],0.952406


In [423]:
data_true.loc[data_true['attitudes_count'].sort_values(ascending=False)[:10].index, ['user.screen_name', 'clean_text', 'attitudes_count']]

,user.screen_name,clean_text,attitudes_count
20939,何炅,吴亦凡好有趣一男的。,30891
74001,舒淇,肚子暖暖 心就暖 [色][色][色],29275
90481,包贝尔,饿了，你吃啥呢？,14388
25295,BeatsbyDre,朋友，吃面吗？又长又宽的那种👇,1287
52621,鹿透社,鹿晗也发歌了，而且还是认真用心做的好歌，希望你朋友圈的微商每次找你转发的时候也能回馈一下，哦...,709
72937,Clear清扬,清扬能让你头发无懈可击，这也确是我本意！明天的演唱会，我在南京等你！Ah wu ah nah...,556
1410,闫紫境GwAwa,这就是hiphop！Hiphop的精神最重要！[good][good][good],555
25552,湖南卫视七十二层奇楼,我听这歌又酷又甜[心] 凡凡 好久不见甚是想念～,549
19120,限定热狗丨思聪,吃了这碗面🍜我们就是一家人[加油],418
20671,西西里岛岛主金女士,哈哈哈哈哈哈哈哈哈他真的好可爱哦！！！！！！！！！,318


In [424]:
data_true.loc[data_true['attitudes_count'].sort_values(ascending=False)[:100].index, ['user.screen_name', 'clean_text', 'attitudes_count']].sample(5)

,user.screen_name,clean_text,attitudes_count
32735,小麦粉,怪不得說克莉絲吳國際巨星呢，唱跳能力態度不說，這次公關團隊水準確實跟上了，可以,13
8257,alex卞正伟,SkrSkr~这波很SKR～ 改天，我也写个《小碗细面》，来个重庆麻辣味[酷],28
29983,花魁今天不接单,mv好可爱啊！！！！还挺好听的！哥哥我爱你[给你小心心],11
13168,moony-whale,真好呀,56
11450,Kiona潜潜潜,,12


### 3. 有多少人拿吴亦凡跟蔡徐坤做对比？

In [425]:
data_true.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96018 entries, 0 to 102117
Data columns (total 22 columns):
attitudes_count           96018 non-null int32
comments_count            96018 non-null int32
reposts_count             96018 non-null int32
mid                       96018 non-null int64
raw_text                  96018 non-null object
source                    96018 non-null object
user.description          96018 non-null object
user.follow_count         96018 non-null int32
user.followers_count      96018 non-null int32
user.gender               96018 non-null object
user.id                   96018 non-null int64
user.mbrank               96018 non-null int32
user.mbtype               96018 non-null int64
user.profile_url          96018 non-null object
user.profile_image_url    96018 non-null object
user.screen_name          96018 non-null object
user.statuses_count       96018 non-null int64
user.urank                96018 non-null int64
user.verified             96018 non-n

In [426]:
data_true['clean_text'].sample(10)

84341    哎！你看这碗它又大又圆！呦！你看这面它又长又宽！
69865               啊啊啊啊啊女神啊啊啊啊啊啊
46546                  [并不简单]校长来了
92157                      瑞思拜瑞思拜
88864                          转发
89383                            
18134                           哇
8378                           转发
40932              额。。。这风格也太像d张伟？
78300              这辈子的梦想就是睡到你[污]
Name: clean_text, dtype: object

In [443]:
data_true['raw_text'].str.contains('kun|坤|律师|球|函|cxk|比|弟|CXK|胸|格局|气度|衬托').sum()

6229

In [442]:
data_kun = data_true.loc[data_true['raw_text'].str.contains('kun|坤|律师|球|函|cxk|比|弟|CXK|胸|格局|气度|衬托'),
             ['user.screen_name', 'raw_text', 'attitudes_count']]

In [429]:
data_kun.loc[list(data_kun['attitudes_count'].nlargest(10).index), ['raw_text', 'attitudes_count']]

,raw_text,attitudes_count
92800,现在艺人公关真的是很厉害，能把嘲点转换为自己的亮点，就比如这个大碗宽面，这么一发想嘲的人可能...,43
27741,凡凡趁着坤坤这波居然完美洗白了？？关键的是歌还不错？！//@藤新Jiven:老吳這個可以，哈...,42
1329,妙妙妙！大气又可爱！才华横溢我的大凡凡！瑞思拜[米妮爱你]#吴亦凡[超话]# [米奇比心][...,37
82171,自黑新高度[允悲]这心态不得不佩服！我凡真不是普通人的气度，爱了爱了！永远支持你呀！[心][...,21
30017,出大事了 cxk要g了吗，凡少也开始玩怪东西了,20
64918,和律师函相比真的是高下立判了,17
84339,还是凡凡格局大[赞]cxk就是个弟弟,16
5722,我的宝贝今天真的好棒[羞嗒嗒] 顺便瞎说一番 今天打开空间朋友圈全部都是对不起吴亦凡的发言 ...,12
30014,我也欠吴老师一个道歉，起码是条会打篮球的刚烈汉子。,12
50951,天蝎气场让人觉得有距离且紧绷给人霸道的假象，好多天蝎明星都经历过被人diss人设太霸道总裁太...,11


In [440]:
data_true[data_true['raw_text']!=''].shape

(96018, 22)

In [441]:
6229/96018

0.06487325293174197

### 4. 有多少人开始路转粉了？

In [432]:
data_true['clean_text'].sample(10)

30801                                                     
65069                                                     
59486                                                 转发微博
5111                                                   我哭了
55020                                               黑转粉？？？
28544                                         撕葱真的虽迟但到[允悲]
81668                                                   轻松
43137                                                聪虽迟但到
39631    51真的太可爱了！！不是不care大家的调侃 但不恼羞成怒 默默酝酿得出最优解。真的太帅了r...
41557                                                     
Name: clean_text, dtype: object

In [433]:
data_true['raw_text'].str.contains('转粉|爱上|重新|路|圈粉|espect|瑞思拜').sum()

3646

In [434]:
data_true.loc[data_true['raw_text'].str.contains('转粉|爱上|重新|路|圈粉|espect|瑞思拜'),
             ['user.screen_name', 'raw_text']].sample(5)

,user.screen_name,raw_text
50755,吾细渣渣帆,不得不说重新认识了吴亦凡，，，，好听好听
5592,海水wei甜,快乐就完事儿了🍜 Respect👊
27477,此号不废弃敬请留存,路转粉… [喵喵][喵喵][喵喵]
36350,齐leyy,瑞思拜
40458,不能sang要加油,这个被疯狂玩梗还作出相应作品的吴亦凡真的让人圈粉。希望坤坤也能一笑置之他的那个恶俗梗，虽说他...


In [435]:
fans = data_true.loc[data_true['raw_text'].str.contains('转粉|爱上|重新|路|圈粉|espect|瑞思拜'), 
                     'user.gender'].value_counts()

In [436]:
bar = Bar("路转粉的男女性别比例", width = 600,height=500)
bar.add("", ['女', '男'], fans.values, is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

In [437]:
fans/fans.sum()

f    0.669501
m    0.330499
Name: user.gender, dtype: float64

### 5. 评论的词云图

In [438]:
import jieba
from collections import Counter
from pyecharts import WordCloud

jieba.add_word('吴亦凡')
jieba.add_word('蔡徐坤')
jieba.add_word('ikun')
jieba.add_word('凡凡')
jieba.add_word('Kris')

swords = [x.strip() for x in open ('stopwords.txt','r',encoding='utf-8')]

In [439]:
def plot_word_cloud(data, swords):
    text = ''.join(data)
    words = list(jieba.cut(text))
    ex_sw_words = []
    for word in words:
        if len(word)>1 and (word not in swords):
            ex_sw_words.append(word)
    c = Counter()
    c = Counter(ex_sw_words)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    wordcloud = WordCloud(width=1300, height=620)
    wordcloud.add("", wc_data['word'], wc_data['counts'], word_size_range=[20, 100])
    return wordcloud

In [132]:
plot_word_cloud(data=data_true.loc[-data_true['clean_text'].str.contains("转发"), 'clean_text'], swords=swords)